# Graded Exercise #3

[EuroSAT](https://zenodo.org/records/7711810#.ZAm3k-zMKEA) is a land use and land cover classification dataset. The dataset is based on Sentinel-2 satellite imagery covering 13 spectral bands and consists of 10 Land Use and Land Cover (LULC) classes with a total of 27,000 labeled and geo-referenced images. 

Using the following code, you can create a DataFrame that includes columns detailing the center locations of the images and their corresponding Land Use and Land Cover (LULC) classes.

Our task: Perform **supervised Machine Learning (ML)** on this dataset. Later we will use **Dask** in your implementation. We will follow the following key steps:
- Create relevant features and analyse them using some visualizations and statistical tools. You can start with features representing the mean and range of spectral bands in these images. You are free to explore more relevant features.
- Split the dataset into training, validation, and test sets.
- Choose an appropriate ML algorithm.
- Train and assess the model's performance.
- Adjust the model's hyperparameters to optimize its performance.

## 1. Creating the work space
Import all the relevant folders and include the file plath to the where the imagery data. 

In [2]:
import numpy as np
import rasterio
import rasterio.features
import rasterio.warp
import geojson
import matplotlib.pyplot as plt
from scipy import stats

In [3]:
import os
from os.path import isfile, join
import pandas as pd

In [4]:
directory_path = 'EuroSAT_MS/'

## 2. Creating target variable data frame
The goal of this excersize is to work on our understanding of machine learning concepts and try to put it into practise. Using the imagery provided by the professor we will create a df with the lat, long and respective land use classification. Land use classification is our target variable and the readings from the bands, as well as the derived features, will be used to create a model that would accurate identify land use classification in a new set up imager.

The code given in this section gets the center coordiante from images. The coordiantes of the center of images makes up our data set. It is at these locations we'll be looking at the band readings and trying to predict land use.  

In [6]:
# function to get the center coordinate of the image
def get_cent(filename):
    with rasterio.open(filename) as dataset:
        # Read the dataset's valid data mask as a ndarray.
        mask = dataset.dataset_mask()

        # Extract feature shapes and values from the array.
        for geom, val in rasterio.features.shapes(
                mask, transform=dataset.transform):

            # Transform shapes from the dataset's own coordinate
            # reference system to CRS84 (EPSG:4326).
            geom = rasterio.warp.transform_geom(
                dataset.crs, 'EPSG:4326', geom, precision=6)
            ls = list(geojson.utils.coords(geom))
            x = []
            y = []
            for row in ls:
                x.append(row[0])
                y.append(row[1])
            cent = [min(y)+(max(y)-min(y))/2,min(x)+(max(x)-min(x))/2]
    return cent

In [7]:
subfolders = [ f.path for f in os.scandir(directory_path) if f.is_dir() ]

In [8]:
df = pd.DataFrame(columns=["Lat", "Lon", "Class"])
for i in range(len(subfolders)):
    image_path = subfolders[i]
    class_name = os.path.basename(image_path)
    all_images = [f for f in os.listdir(image_path) if os.path.isfile(join(image_path, f))]
    print(class_name,len(all_images))
    for j in range(len(all_images)):
        cent = get_cent(image_path+'/'+all_images[j])
        new_row = pd.DataFrame({"Lat": cent[0], "Lon": cent[1], "Class": class_name}, index=[0])
        df = pd.concat([df, new_row], ignore_index=True)
        

AnnualCrop 3000
Forest 3000
HerbaceousVegetation 3000
Highway 2500
Industrial 2500
Pasture 2000
PermanentCrop 2500
Residential 3000
River 2500
SeaLake 3000


In [9]:
df

,Lat,Lon,Class
0,44.035220,28.559055,AnnualCrop
1,39.085801,-1.829726,AnnualCrop
2,48.977294,4.239720,AnnualCrop
3,48.892610,4.089878,AnnualCrop
4,51.832851,18.084961,AnnualCrop
...,...,...,...
26995,42.124618,12.194484,SeaLake
26996,59.256125,15.611878,SeaLake
26997,51.688627,4.137389,SeaLake
26998,52.506092,8.335339,SeaLake


## 2. Creating relevant features 
Right now our data frame only contains the dependent variable, but no independent variables yet. We need to collect and create variables from the imagery to build out model. We wanted to include the max, median, range, and mode of all the band readings to have a broad understanding of what is going on at each observation point (centre coordinate). While the band readings are nice, we do not need to let the machine do all the work. There are several indexes that have been proven to help identify land use classification. The following indexes will be created as additional features: 

1.	Normalized Difference Vegetation Index (NDVI)- this versatile index is used in agriculture, natural hazards such as landslides, land use/land cover change detection, environmental monitoring, water resources etc. to name a few. NDVI provides valuable information in wide range of applications making it an important feature to be studied.
NDVI = (B8-B4) / (B8+B4).

2.	SAVI- Soil Adjusted Vegetation Index (SAVI) is used to correct Normalized Difference Vegetation Index (NDVI) for the influence of soil brightness in areas where vegetative cover is low. The higher the NDVI values (the same stands for SAVI) the denser (and healthier) the vegetation. But NDVI start saturating after the value of 0.7, while SAVI at this point is only 0.3. This means that SAVI can be better used in dense vegetation because it saturates less fast. 
For Sentinel-2 the formula is:
(B08 - B04) / (B08 + B04 + L) * (1.0 + L); L = 0.428
where: L is a soil brightness correction factor ranging from 0 to 1
L = 1 low vegetation cover, L = 0 high vegetation cover, L = 0.5 intermediate vegetation cover.
https://custom-scripts.sentinel-hub.com/custom-scripts/sentinel-2/savi/

3.	Normalised difference water index (NDWI)- is used to highlight open water features in a satellite image, allowing a water body to “stand out” against the soil and vegetation. The downside of the index is that it is sensitive to built structures, which can lead to overestimation of water bodies.
For Sentinel 2 data:
NDWI= (Band 3 – Band 8)/(Band 3 + Band 8)
NDWI: Index Formula, Value Range, And Uses In Agriculture (eos.com)


In [12]:
#extract features
def extract_spectral_features(filename):
    with rasterio.open(filename) as dataset:
        # Read all spectral bands
        bands = [dataset.read(band) for band in range(1, dataset.count + 1)]

        # Initialize lists to store statistics
        band_stats = []

        for band in bands:
            # Calculate mean and range for each band
            mean = np.nanmean(band)  # Calculate mean, handling NaN values
            min_val = np.nanmin(band)  # Calculate minimum value, handling NaN values
            max_val = np.nanmax(band)  # Calculate maximum value, handling NaN values
            median = np.nanmedian(band) # Calculate median, handling NaN values
            band_range = max_val - min_val

            band_stats.append((mean,median, band_range))

    return band_stats

## 3. Split the data in training, test, and validation
For our work it was important to have separate sets for training, validation, and testing. The primary reason was to reduce over fitting. The separation between these sets ensures that the model can perform well in real-world scenarios. Using the same data for all the three purposes can lead to over fitting as the model will simply memorize the data rather than making meaning results. We chose to do kmeans cross validations for our work as it is less biased than the simple train/test/valid split. This method of spliting data will ideally not result in overfitting, while still being relativelly simple to implement. 


In [3]:
from sklearn.model_selection import KFold

In [ ]:
X = np.array([[1, 2], [3, 4], [1, 2], [3, 4]])
y = np.array([1, 2, 3, 4])
kf = KFold(n_splits=10) 
kf.get_n_splits(X)

## 4. Developing our model
Random forest is a well-known machine learning model, commonly used for classification tasks. In recent studies random forest model was found to out preform artificial neural networks with the same task of land use classification [1] When working with Sentinel-2 specifically, as we are here, random forest was found to be the stand out model for land use/land cover classification [2]. For these reasons we chose random forest as our model.

[1] https://www.frontiersin.org/articles/10.3389/frai.2022.964279/full#:~:text=We%20classified%20land%20use%20and,conducted%20by%20Tan%20et%20al.

[2] Ge, G., Shi, Z., Zhu, Y., Yang, X., & Hao, Y. (2020). Land use/cover classification in an arid desert-oasis mosaic landscape of China using remote sensed imagery: Performance assessment of four machine learning algorithms. Global Ecology and Conservation, 22, e00971. https://www.sciencedirect.com/science/article/pii/S2351989420300202

**Points of discussion**:
- Can you explain the criteria and rationale behind the features you created? What other features you would select from these images in addition to the mean and range?
   
- Why is it important to have separate sets for training, validation, and testing? Which split did you consider and why?
    
- What factors influenced your choice of a specific machine learning algorithm?
    
- How did hyperparameter tuning impact the model's performance, and what were the final hyperparameter settings?
- What is the impact of using DASK to solve this problem? What is the impact of changing DASK parameters like chunk size? You may consider checking CPU, memory usage, processing time, ...